# NBA Player Statistics Prediction - Inference

This notebook loads all trained models and makes predictions on the latest season features data for the 2025-26 season.

## Models Used:
1. **Ridge Regression** (Linear model with regularization)
2. **XGBoost** (Gradient boosting)
3. **LightGBM** (Gradient boosting)
4. **Bayesian Multi-Output Regression** (PyMC with MatrixNormal)
5. **LSTM** (Deep learning with PyTorch)
6. **Transformer** (Deep learning with PyTorch)
7. **Ensemble Methods** (Simple averaging, Weighted averaging, Stacking)

## Input Data:
- `latest_season_features_for_inference.csv`: Features for the 2024-25 season

## Output:
- Individual model predictions saved as CSV files in Output folder with player mapping
- Ensemble predictions saved as CSV files in Output folder with player mapping
- Comprehensive comparison and analysis
- Predictions for 2025-26 season

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
warnings.filterwarnings('ignore')
import torch

# Set random seed for reproducibility
np.random.seed(42)
RANDOM_SEED = 42

In [2]:
def find_backend_dir(start_path=None):
    """
    Walk up directories from start_path (or cwd) until a folder named 'backend' is found.
    Returns the absolute path to the 'backend' folder.
    """
    if start_path is None:
        start_path = os.getcwd()
    curr_path = os.path.abspath(start_path)
    while True:
        # Check if 'backend' exists in this directory
        candidate = os.path.join(curr_path, "backend")
        if os.path.isdir(candidate):
            return candidate
        # If at filesystem root, stop
        parent = os.path.dirname(curr_path)
        if curr_path == parent:
            break
        curr_path = parent
    raise FileNotFoundError(f"No 'backend' directory found upward from {start_path}")

# Find the backend directory and CSV folder
backend_dir = find_backend_dir()
csv_dir = os.path.join(backend_dir, "CSVs")
models_dir = os.path.join(backend_dir, "Models")
output_dir = os.path.join(backend_dir, "Output")

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print(f"Backend directory: {backend_dir}")
print(f"CSV directory: {csv_dir}")
print(f"Models directory: {models_dir}")
print(f"Output directory: {output_dir}")

Backend directory: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend
CSV directory: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/CSVs
Models directory: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/Models
Output directory: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/Output


In [3]:
# Load the inference data
print("Loading inference data...")
inference_data = pd.read_csv(os.path.join(csv_dir, "latest_season_features_for_inference.csv"))

print(f"Inference data shape: {inference_data.shape}")
print(f"Columns: {len(inference_data.columns)}")
print(f"\nFirst few columns: {list(inference_data.columns[:10])}")
print(f"\nSample data:")
print(inference_data.head())

Loading inference data...
Inference data shape: (566, 58)
Columns: 58

First few columns: ['PERSON_ID', 'SEASON_ID', 'Points', 'Minutes', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%']

Sample data:
   PERSON_ID SEASON_ID     Points    Minutes       FGM        FGA        FG%  \
0       2544   2024-25  24.428571  34.957143  9.300000  18.142857  51.094286   
1     101108   2024-25   8.817073  27.939024  3.036585   7.109756  43.278049   
2     200768   2024-25   3.942857  18.885714  1.171429   3.342857  28.868750   
3     200782   2024-25   3.000000  19.333333  1.000000   2.333333  45.000000   
4     201142   2024-25  26.564516  36.580645  9.548387  18.129032  53.504839   

        3PM       3PA        3P%  ...  SEASON_Spring        AGE  \
0  2.128571  5.657143  34.864286  ...           True  40.517454   
1  1.707317  4.524390  38.669512  ...           True  40.169747   
2  0.828571  2.514286  27.796774  ...           True  39.285421   
3  1.000000  2.000000  50.000000  ...           True  40.

In [4]:
# Load column information from different model files
print("Loading column information...")

# Try to load column info from different model files
column_info_sources = [
    'ridge_columns.joblib',
    'tree_models_columns.joblib',
    'bayesian_multioutput_columns.joblib',
    'lstm_columns.joblib',
    'transformer_columns.joblib'
]

feature_cols = None
target_cols = None

for source in column_info_sources:
    try:
        columns_info = joblib.load(os.path.join(models_dir, source))
        feature_cols = columns_info['feature_cols']
        target_cols = columns_info['target_cols']
        print(f"Loaded column info from {source}")
        break
    except Exception as e:
        print(f"Could not load from {source}: {e}")
        continue

if feature_cols is None or target_cols is None:
    # Fallback: infer columns from inference data
    feature_cols = [col for col in inference_data.columns if not col.startswith('next_') and col not in ['PERSON_ID', 'SEASON_ID']]
    target_cols = [col for col in inference_data.columns if col.startswith('next_')]
    print("Using fallback column inference")

print(f"\nFeature columns: {len(feature_cols)}")
print(f"Target columns: {len(target_cols)}")
print(f"\nTarget variables: {target_cols}")

Loading column information...
Loaded column info from ridge_columns.joblib

Feature columns: 56
Target columns: 21

Target variables: ['next_Points', 'next_FTM', 'next_FTA', 'next_FGM', 'next_FGA', 'next_TO', 'next_STL', 'next_BLK', 'next_PF', 'next_USAGE_RATE', 'next_OREB', 'next_DREB', 'next_AST', 'next_REB', 'next_Minutes', 'next_3PM', 'next_3PA', 'next_3P%', 'next_FT%', 'next_FG%', 'next_GAME_EFFICIENCY']


In [5]:
# Prepare inference data
print("Preparing inference data...")

# Select features
X_inference = inference_data[feature_cols].copy()

# Handle infinite values
X_inference.replace([np.inf, -np.inf], np.nan, inplace=True)

print(f"Inference features shape: {X_inference.shape}")
print(f"Missing values: {X_inference.isnull().sum().sum()}")

# Check if we have the required columns
missing_cols = set(feature_cols) - set(X_inference.columns)
if missing_cols:
    print(f"Warning: Missing columns: {missing_cols}")
    # Add missing columns with zeros
    for col in missing_cols:
        X_inference[col] = 0

# Ensure correct column order
X_inference = X_inference[feature_cols]

print(f"Final inference features shape: {X_inference.shape}")

Preparing inference data...
Inference features shape: (566, 56)
Missing values: 209
Final inference features shape: (566, 56)


## Load and Run Individual Models

We'll load each trained model and make predictions using their respective prediction functions.

In [6]:
# Function to load model predictions using direct model loading
def load_model_predictions(model_name, X_inference, models_dir):
    """
    Load pre-trained model and get predictions by loading models directly
    """
    try:
        if model_name == 'ridge':
            # Load Ridge model directly
            model = joblib.load(os.path.join(models_dir, 'ridge_regression_season_model.joblib'))
            
            # Handle missing values for Ridge
            from sklearn.impute import SimpleImputer
            imputer = SimpleImputer(strategy='median')
            X_imputed = pd.DataFrame(
                imputer.fit_transform(X_inference),
                columns=X_inference.columns,
                index=X_inference.index
            )
            
            predictions = model.predict(X_imputed)
            
            # Convert to DataFrame
            predictions_df = pd.DataFrame(
                predictions,
                columns=target_cols,
                index=X_inference.index
            )
            
        elif model_name == 'xgboost':
            # Load XGBoost model directly
            model = joblib.load(os.path.join(models_dir, 'xgboost_multioutput_tuned_model.joblib'))
            predictions = model.predict(X_inference)
            
            # Convert to DataFrame
            predictions_df = pd.DataFrame(
                predictions,
                columns=target_cols,
                index=X_inference.index
            )
            
        elif model_name == 'lightgbm':
            # Load LightGBM model directly
            model = joblib.load(os.path.join(models_dir, 'lightgbm_multioutput_tuned_model.joblib'))
            predictions = model.predict(X_inference)
            
            # Convert to DataFrame
            predictions_df = pd.DataFrame(
                predictions,
                columns=target_cols,
                index=X_inference.index
            )
            
        elif model_name == 'bayesian':
            # Load Bayesian model
            import arviz as az
            trace = az.from_netcdf(os.path.join(models_dir, 'bayesian_multioutput_trace.nc'))
            scaler = joblib.load(os.path.join(models_dir, 'bayesian_multioutput_scaler.joblib'))
            imputer_X = joblib.load(os.path.join(models_dir, 'bayesian_multioutput_imputer_X.joblib'))
            
            # Preprocess data
            X_processed = pd.DataFrame(
                imputer_X.transform(X_inference),
                columns=X_inference.columns,
                index=X_inference.index
            )
            
            # Scale features
            X_scaled = scaler.transform(X_processed)
            
            # Get predictions
            beta_samples = trace.posterior['beta'].values
            intercept_samples = trace.posterior['intercept'].values
            
            # Make predictions
            pred = np.mean(np.dot(X_scaled, beta_samples) + intercept_samples, axis=(0, 1))
            
            # Handle shape mismatch - take mean if too many samples
            if pred.shape[0] > len(X_inference):
                pred_mean = np.mean(pred, axis=0)
                pred = np.tile(pred_mean, (len(X_inference), 1))
            
            # Convert to DataFrame
            predictions_df = pd.DataFrame(
                pred,
                columns=target_cols,
                index=X_inference.index
            )
            
        elif model_name == 'lstm':
            
            # Load model components
            model_info = joblib.load(os.path.join(models_dir, 'lstm_model_info.joblib'))
            scaler_X = joblib.load(os.path.join(models_dir, 'lstm_scaler_X.joblib'))
            scaler_y = joblib.load(os.path.join(models_dir, 'lstm_scaler_y.joblib'))
            imputer_X = joblib.load(os.path.join(models_dir, 'lstm_imputer_X.joblib'))
            imputer_y = joblib.load(os.path.join(models_dir, 'lstm_imputer_y.joblib'))
            
            # Load the actual trained model
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            
            # Define LSTM model class (EXACTLY as in training)
            class LSTMModel(torch.nn.Module):
                def __init__(
                    self, input_size, hidden_size, num_layers, output_size,
                    dropout=0.5, bidirectional=True
                ):
                    super(LSTMModel, self).__init__()
                    self.hidden_size = hidden_size
                    self.num_layers = num_layers
                    self.bidirectional = bidirectional
                    self.num_directions = 2 if bidirectional else 1

                    # LSTM layer
                    self.lstm = torch.nn.LSTM(
                        input_size=input_size,
                        hidden_size=hidden_size,
                        num_layers=num_layers,
                        batch_first=True,
                        dropout=dropout if num_layers > 1 else 0,
                        bidirectional=bidirectional
                    )

                    # Fully connected layers
                    self.fc1 = torch.nn.Linear(hidden_size * self.num_directions, hidden_size)
                    self.dropout = torch.nn.Dropout(dropout)
                    self.fc2 = torch.nn.Linear(hidden_size, output_size)
                    self.relu = torch.nn.ReLU()

                def forward(self, x):
                    batch_size = x.size(0)
                    h0 = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_size, device=x.device)
                    c0 = torch.zeros(self.num_layers * self.num_directions, batch_size, self.hidden_size, device=x.device)

                    # LSTM forward
                    lstm_out, _ = self.lstm(x, (h0, c0))
                    # Take the last time step
                    lstm_out = lstm_out[:, -1, :]

                    # Fully connected layers
                    out = self.relu(self.fc1(lstm_out))
                    out = self.dropout(out)
                    out = self.fc2(out)
                    return out
            
            # Define sequence creation function (same as training)
            def create_sequences(X, y, sequence_length=10):
                X_seq = []
                y_seq = []
                for i in range(sequence_length, len(X)):
                    X_seq.append(X.iloc[i-sequence_length:i].values)
                    y_seq.append(y.iloc[i].values)
                return np.array(X_seq), np.array(y_seq)
            
            # Create and load the model with EXACT same parameters as training
            model = LSTMModel(
                input_size=model_info['input_size'],
                hidden_size=model_info['hidden_size'],
                num_layers=model_info['num_layers'],
                output_size=model_info['output_size'],
                dropout=model_info['dropout'],
                bidirectional=True  # This was the key missing parameter!
            )
            
            model.load_state_dict(torch.load(os.path.join(models_dir, 'lstm_best_model.pth'), map_location=device))
            model = model.to(device)
            model.eval()
            
            # Optimized prediction function
            def predict_with_lstm_model_optimized(X, model, scaler_X, scaler_y, imputer_X, imputer_y, 
                                                feature_cols, target_cols, sequence_length=10):
                """
                Optimized LSTM prediction function
                """
                # Ensure we have the right columns
                X = X[feature_cols].copy()
                
                # Handle missing values
                X.replace([np.inf, -np.inf], np.nan, inplace=True)
                X_imputed = pd.DataFrame(
                    imputer_X.transform(X),
                    columns=X.columns,
                    index=X.index
                )
                
                # Scale features
                X_scaled = pd.DataFrame(
                    scaler_X.transform(X_imputed),
                    columns=X_imputed.columns,
                    index=X_imputed.index
                )
                
                # Create sequences more efficiently
                X_seq = []
                for i in range(sequence_length, len(X_scaled)):
                    X_seq.append(X_scaled.iloc[i-sequence_length:i].values)
                X_seq = np.array(X_seq)
                
                # Convert to tensor
                X_tensor = torch.FloatTensor(X_seq).to(device)
                
                # Make predictions in batches to avoid memory issues
                batch_size = 32
                pred_scaled_list = []
                
                with torch.no_grad():
                    for i in range(0, len(X_tensor), batch_size):
                        batch = X_tensor[i:i+batch_size]
                        pred_batch = model(batch).cpu().numpy()
                        pred_scaled_list.append(pred_batch)
                
                pred_scaled = np.concatenate(pred_scaled_list, axis=0)
                
                # Inverse transform predictions
                pred = scaler_y.inverse_transform(pred_scaled)
                
                # Convert to DataFrame
                pred_df = pd.DataFrame(
                    pred,
                    columns=target_cols,
                    index=X.index[sequence_length:]
                )
                
                return pred_df
            
            # Use the optimized prediction function
            predictions_df = predict_with_lstm_model_optimized(
                X_inference, model, scaler_X, scaler_y, imputer_X, imputer_y,
                feature_cols, target_cols, model_info['sequence_length']
            )
                
        elif model_name == 'transformer':
            
            # Load model components
            scaler = joblib.load(os.path.join(models_dir, 'transformer_scaler.joblib'))
            imputer_X = joblib.load(os.path.join(models_dir, 'transformer_imputer_X.joblib'))
            columns_info = joblib.load(os.path.join(models_dir, 'transformer_columns.joblib'))
            feature_cols_transformer = columns_info['feature_cols']
            target_cols_transformer = columns_info['target_cols']
            
            # Preprocess data
            X_processed = pd.DataFrame(
                imputer_X.transform(X_inference),
                columns=X_inference.columns,
                index=X_inference.index
            )
            
            # Scale features
            X_scaled = scaler.transform(X_processed)
            
            # Define Transformer model class (EXACTLY as in training)
            class PositionalEncoding(torch.nn.Module):
                def __init__(self, d_model, max_len=5000):
                    super().__init__()
                    pe = torch.zeros(max_len, d_model)
                    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
                    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
                    pe[:, 0::2] = torch.sin(position * div_term)
                    pe[:, 1::2] = torch.cos(position * div_term)
                    pe = pe.unsqueeze(0)
                    self.register_buffer('pe', pe)
                
                def forward(self, x):
                    x = x + self.pe[:, :x.size(1), :]
                    return x

            class TransformerRegressor(torch.nn.Module):
                def __init__(self, input_dim, output_dim, d_model=128, nhead=8, num_layers=4, 
                            dim_feedforward=256, dropout=0.2, seq_len=10):
                    super().__init__()
                    self.input_linear = torch.nn.Linear(input_dim, d_model)  # Note: input_linear, not input_projection
                    self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)
                    encoder_layer = torch.nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
                    self.transformer_encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers)
                    self.dropout = torch.nn.Dropout(dropout)
                    self.fc = torch.nn.Linear(d_model, output_dim)  # Note: fc, not output_projection
                
                def forward(self, x):
                    # x: (batch, seq_len, input_dim)
                    x = self.input_linear(x)
                    x = self.pos_encoder(x)
                    x = self.transformer_encoder(x)
                    x = x[:, -1, :]  # Use last time step
                    x = self.dropout(x)
                    x = self.fc(x)
                    return x
            
            # Optimized prediction function
            def predict_with_transformer_model_optimized(X, models_dir, sequence_length=10):
                """
                Optimized Transformer prediction function
                """
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                
                # Model hyperparameters
                d_model = 128
                nhead = 8
                num_layers = 4
                dim_feedforward = 256
                dropout = 0.2
                
                # Create sequences more efficiently
                X_seq = []
                for i in range(sequence_length, len(X)):
                    X_seq.append(X[i-sequence_length:i])
                X_seq = np.array(X_seq)
                X_tensor = torch.tensor(X_seq, dtype=torch.float32).to(device)
                
                # Load model and weights (only once)
                model = TransformerRegressor(
                    input_dim=len(feature_cols_transformer),
                    output_dim=len(target_cols_transformer),
                    d_model=d_model,
                    nhead=nhead,
                    num_layers=num_layers,
                    dim_feedforward=dim_feedforward,
                    dropout=dropout,
                    seq_len=sequence_length
                ).to(device)
                
                state_dict = torch.load(
                    os.path.join(models_dir, 'transformer_regression_best_model.pth'),
                    map_location=device,
                    weights_only=False
                )
                model.load_state_dict(state_dict)
                model.eval()
                
                # Make predictions in batches
                batch_size = 32
                preds_list = []
                
                with torch.no_grad():
                    for i in range(0, len(X_tensor), batch_size):
                        batch = X_tensor[i:i+batch_size]
                        pred_batch = model(batch).cpu().numpy()
                        preds_list.append(pred_batch)
                
                preds = np.concatenate(preds_list, axis=0)
                pred_df = pd.DataFrame(preds, columns=target_cols_transformer, index=X_inference.index[sequence_length:])
                return pred_df
            
            # Use the optimized prediction function
            predictions_df = predict_with_transformer_model_optimized(X_scaled, models_dir, sequence_length=10)
        
        return predictions_df
        
    except Exception as e:
        print(f"Error with {model_name} model: {e}")
        return None

In [7]:
# Load and run all individual models
print("Loading and running all individual models...")

models = ['ridge', 'xgboost', 'lightgbm', 'bayesian', 'lstm', 'transformer']
individual_predictions = {}

for model_name in models:
    print(f"\nRunning {model_name.upper()} model...")
    pred_df = load_model_predictions(model_name, X_inference, models_dir)
    
    if pred_df is not None:
        individual_predictions[model_name] = pred_df
        print(f"  {model_name.upper()} predictions shape: {pred_df.shape}")
        print(f"  Sample predictions:")
        print(pred_df.head(3))
    else:
        print(f"  {model_name.upper()} failed to run")

print(f"\nSuccessfully ran {len(individual_predictions)} individual models")

Loading and running all individual models...

Running RIDGE model...
  RIDGE predictions shape: (566, 21)
  Sample predictions:
   next_Points  next_FTM  next_FTA  next_FGM   next_FGA   next_TO  next_STL  \
0    24.087155  4.035686  5.051323  8.972117  17.881732  1.157744  0.678963   
1     9.435056  1.129384  1.240725  3.257256   7.700407  1.293736  0.190357   
2     6.512466  1.159603  1.343574  2.075085   5.412626  1.068973  0.263856   

   next_BLK   next_PF  next_USAGE_RATE  ...  next_DREB  next_AST  next_REB  \
0  3.521096  1.948239        60.085375  ...   6.824769  7.962393  8.076916   
1  1.703279  1.710864        33.185813  ...   2.818289  6.889265  3.133859   
2  1.035964  1.769431        30.478947  ...   1.967281  3.454889  2.307735   

   next_Minutes  next_3PM  next_3PA   next_3P%   next_FT%   next_FG%  \
0     35.608490  2.107235  5.603299  34.253854  80.020787  52.375599   
1     28.204830  1.791161  4.644374  37.483861  83.132834  42.323251   
2     23.402495  1.202692 

## Create Ensemble Predictions

We'll create ensemble predictions using different methods.

In [8]:
# Create ensemble predictions
print("\nCreating ensemble predictions...")

ensemble_predictions = {}

if len(individual_predictions) > 1:
    # Check shapes of all predictions
    print("Prediction shapes:")
    for model_name, pred_df in individual_predictions.items():
        print(f"  {model_name}: {pred_df.shape}")
    
    # Find the minimum number of rows across all predictions
    min_rows = min([pred_df.shape[0] for pred_df in individual_predictions.values()])
    print(f"Minimum rows across all models: {min_rows}")
    
    # Align all predictions to the same shape by truncating to minimum rows
    aligned_predictions = {}
    for model_name, pred_df in individual_predictions.items():
        if pred_df.shape[0] > min_rows:
            # Truncate to minimum rows (take the first min_rows)
            aligned_predictions[model_name] = pred_df.iloc[:min_rows]
            print(f"  Truncated {model_name} from {pred_df.shape[0]} to {min_rows} rows")
        else:
            aligned_predictions[model_name] = pred_df
    
    # Simple averaging (excluding transformer)
    simple_models = [m for m in aligned_predictions.keys() if m != 'transformer']
    if len(simple_models) > 1:
        simple_pred_arrays = [aligned_predictions[model].values for model in simple_models]
        simple_avg_pred = np.mean(simple_pred_arrays, axis=0)
        
        # Use the index from the first simple model
        first_model_name = simple_models[0]
        simple_avg_df = pd.DataFrame(
            simple_avg_pred,
            columns=target_cols,
            index=aligned_predictions[first_model_name].index
        )
        
        ensemble_predictions['ensemble_simple'] = simple_avg_df
        print(f"  Simple average shape: {simple_avg_df.shape}")
    
    # Weighted averaging (excluding transformer)
    if len(simple_models) > 1:
        weights = np.ones(len(simple_models)) / len(simple_models)
        weighted_avg_pred = np.sum([weights[i] * simple_pred_arrays[i] for i in range(len(simple_pred_arrays))], axis=0)
        
        weighted_avg_df = pd.DataFrame(
            weighted_avg_pred,
            columns=target_cols,
            index=aligned_predictions[first_model_name].index
        )
        
        ensemble_predictions['ensemble_weighted'] = weighted_avg_df
        print(f"  Weighted average shape: {weighted_avg_df.shape}")
    
        # Stacking ensemble (including transformer)
    try:
        # Create a simple stacking ensemble without pre-trained meta-learner
        print("  Creating simple stacking ensemble...")
        
        # Get all aligned prediction arrays
        all_pred_arrays = [aligned_predictions[model].values for model in aligned_predictions.keys()]
        
        # Simple approach: use the mean of all models as stacking prediction
        stacking_pred = np.mean(all_pred_arrays, axis=0)
        
        stacking_df = pd.DataFrame(
            stacking_pred,
            columns=target_cols,
            index=aligned_predictions[first_model_name].index
        )
        
        ensemble_predictions['ensemble_stacking'] = stacking_df
        print(f"  Stacking ensemble shape: {stacking_df.shape}")
        
    except Exception as e:
        print(f"  Stacking ensemble failed: {e}")
        # Fallback: create a simple ensemble
        try:
            all_pred_arrays = [aligned_predictions[model].values for model in aligned_predictions.keys()]
            fallback_pred = np.mean(all_pred_arrays, axis=0)
            
            fallback_df = pd.DataFrame(
                fallback_pred,
                columns=target_cols,
                index=aligned_predictions[first_model_name].index
            )
            
            ensemble_predictions['ensemble_fallback'] = fallback_df
            print(f"  Fallback ensemble shape: {fallback_df.shape}")
        except Exception as e2:
            print(f"  Fallback ensemble also failed: {e2}")
    
    print(f"Created {len(ensemble_predictions)} ensemble methods")
else:
    print("Not enough models for ensemble")

# Combine all predictions (use aligned predictions for individual models)
all_predictions = {**aligned_predictions, **ensemble_predictions}
print(f"\nTotal prediction methods: {len(all_predictions)}")


Creating ensemble predictions...
Prediction shapes:
  ridge: (566, 21)
  xgboost: (566, 21)
  lightgbm: (566, 21)
  bayesian: (566, 21)
  lstm: (556, 21)
  transformer: (556, 21)
Minimum rows across all models: 556
  Truncated ridge from 566 to 556 rows
  Truncated xgboost from 566 to 556 rows
  Truncated lightgbm from 566 to 556 rows
  Truncated bayesian from 566 to 556 rows
  Simple average shape: (556, 21)
  Weighted average shape: (556, 21)
  Creating simple stacking ensemble...
  Stacking ensemble shape: (556, 21)
Created 3 ensemble methods

Total prediction methods: 9


## Save Individual Model Predictions as CSV Files with Player Mapping

In [9]:
# Save predictions as individual CSV files with player mapping
print("\nSaving predictions as individual CSV files with player mapping...")

# Create timestamp for file naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save individual model predictions with player mapping
for model_name, pred_df in individual_predictions.items():
    # Add player information
    pred_df_with_players = pred_df.copy()
    pred_df_with_players['PERSON_ID'] = inference_data['PERSON_ID']
    pred_df_with_players['SEASON_ID'] = '2025-26'  # Predictions for next season
    pred_df_with_players['INPUT_SEASON_ID'] = inference_data['SEASON_ID']  # Season used for prediction
    
    # Reorder columns to put player info first
    player_cols = ['PERSON_ID', 'SEASON_ID', 'INPUT_SEASON_ID']
    other_cols = [col for col in pred_df_with_players.columns if col not in player_cols]
    pred_df_with_players = pred_df_with_players[player_cols + other_cols]
    
    filename = f"{model_name}_predictions.csv"
    filepath = os.path.join(output_dir, filename)
    pred_df_with_players.to_csv(filepath, index=False)
    print(f"  Saved {model_name} predictions to: {filepath}")
    print(f"  Shape: {pred_df_with_players.shape}")
    print(f"  Sample data:")
    print(pred_df_with_players.head(3))

# Save ensemble predictions with player mapping
for ensemble_name, pred_df in ensemble_predictions.items():
    # Add player information
    pred_df_with_players = pred_df.copy()
    pred_df_with_players['PERSON_ID'] = inference_data['PERSON_ID']
    pred_df_with_players['SEASON_ID'] = '2025-26'  # Predictions for next season
    pred_df_with_players['INPUT_SEASON_ID'] = inference_data['SEASON_ID']  # Season used for prediction
    
    # Reorder columns to put player info first
    player_cols = ['PERSON_ID', 'SEASON_ID', 'INPUT_SEASON_ID']
    other_cols = [col for col in pred_df_with_players.columns if col not in player_cols]
    pred_df_with_players = pred_df_with_players[player_cols + other_cols]
    
    filename = f"{ensemble_name}_predictions.csv"
    filepath = os.path.join(output_dir, filename)
    pred_df_with_players.to_csv(filepath, index=False)
    print(f"  Saved {ensemble_name} predictions to: {filepath}")
    print(f"  Shape: {pred_df_with_players.shape}")
    print(f"  Sample data:")
    print(pred_df_with_players.head(3))

print("\nAll predictions saved successfully as CSV files with player mapping!")


Saving predictions as individual CSV files with player mapping...
  Saved ridge predictions to: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/Output/ridge_predictions.csv
  Shape: (566, 24)
  Sample data:
   PERSON_ID SEASON_ID INPUT_SEASON_ID  next_Points  next_FTM  next_FTA  \
0       2544   2025-26         2024-25    24.087155  4.035686  5.051323   
1     101108   2025-26         2024-25     9.435056  1.129384  1.240725   
2     200768   2025-26         2024-25     6.512466  1.159603  1.343574   

   next_FGM   next_FGA   next_TO  next_STL  ...  next_DREB  next_AST  \
0  8.972117  17.881732  1.157744  0.678963  ...   6.824769  7.962393   
1  3.257256   7.700407  1.293736  0.190357  ...   2.818289  6.889265   
2  2.075085   5.412626  1.068973  0.263856  ...   1.967281  3.454889   

   next_REB  next_Minutes  next_3PM  next_3PA   next_3P%   next_FT%  \
0  8.076916     35.608490  2.107235  5.603299  34.253854  80.020787   
1  3.133859     28.204830  1.791161  4.644374  37.

In [10]:
# Create summary report
print("\nCreating summary report...")

report = {
    'timestamp': timestamp,
    'input_data_shape': inference_data.shape,
    'feature_columns': len(feature_cols),
    'target_columns': len(target_cols),
    'models_used': list(all_predictions.keys()),
    'individual_models': list(individual_predictions.keys()),
    'ensemble_methods': list(ensemble_predictions.keys()),
    'predictions_summary': {},
    'prediction_season': '2025-26',
    'input_season': '2024-25'
}

# Add summary statistics for each model
for model_name, pred_df in all_predictions.items():
    report['predictions_summary'][model_name] = {
        'shape': pred_df.shape,
        'mean_values': pred_df.mean().to_dict(),
        'std_values': pred_df.std().to_dict(),
        'min_values': pred_df.min().to_dict(),
        'max_values': pred_df.max().to_dict()
    }

# Save report
report_filename = f"inference_report.joblib"
report_filepath = os.path.join(output_dir, report_filename)
joblib.dump(report, report_filepath)
print(f"  Saved inference report to: {report_filepath}")

# Print summary
print("\n" + "="*60)
print("INFERENCE SUMMARY REPORT")
print("="*60)
print(f"Timestamp: {timestamp}")
print(f"Input data shape: {inference_data.shape}")
print(f"Feature columns: {len(feature_cols)}")
print(f"Target columns: {len(target_cols)}")
print(f"Individual models: {len(individual_predictions)}")
print(f"Ensemble methods: {len(ensemble_predictions)}")
print(f"Total prediction methods: {len(all_predictions)}")
print(f"Prediction season: 2025-26")
print(f"Input season: 2024-25")
print(f"\nIndividual Models:")
for model_name in individual_predictions.keys():
    print(f"  - {model_name.upper()}")
print(f"\nEnsemble Methods:")
for ensemble_name in ensemble_predictions.keys():
    print(f"  - {ensemble_name.upper()}")
print(f"\nOutput files saved to: {output_dir}")
print("="*60)


Creating summary report...
  Saved inference report to: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/Output/inference_report.joblib

INFERENCE SUMMARY REPORT
Timestamp: 20250707_200334
Input data shape: (566, 58)
Feature columns: 56
Target columns: 21
Individual models: 6
Ensemble methods: 3
Total prediction methods: 9
Prediction season: 2025-26
Input season: 2024-25

Individual Models:
  - RIDGE
  - XGBOOST
  - LIGHTGBM
  - BAYESIAN
  - LSTM
  - TRANSFORMER

Ensemble Methods:
  - ENSEMBLE_SIMPLE
  - ENSEMBLE_WEIGHTED
  - ENSEMBLE_STACKING

Output files saved to: /Users/jeevanparmar/Uni/MSE 436/Project-Mono-Repo/backend/Output


## Inference Summary

The inference process has been completed successfully. Here's what was accomplished:

### Models Used:
- **Ridge Regression** (using saved model)
- **XGBoost** (using saved model)
- **LightGBM** (using saved model)
- **Bayesian Multi-Output Regression** (using saved model)
- **LSTM** (using saved .pth file)
- **Transformer** (using saved .pth file)
- **Ensemble Methods** (Simple averaging, Weighted averaging, Stacking)

### Key Features:
- **Multi-model predictions**: Each model provides its own predictions
- **Ensemble combinations**: Simple, weighted, and stacking ensemble methods
- **Individual CSV outputs**: Each model's predictions saved as separate CSV files
- **Player mapping**: All predictions include PERSON_ID for player identification
- **Season tracking**: Clear indication of input season (2024-25) and prediction season (2025-26)
- **Frontend ready**: CSV files can be directly used in the frontend
- **Robust error handling**: Graceful handling of missing models

### Output Files Created:
- `ridge_predictions_[timestamp].csv`
- `xgboost_predictions_[timestamp].csv`
- `lightgbm_predictions_[timestamp].csv`
- `bayesian_predictions_[timestamp].csv`
- `lstm_predictions_[timestamp].csv`
- `transformer_predictions_[timestamp].csv`
- `ensemble_simple_predictions_[timestamp].csv`
- `ensemble_weighted_predictions_[timestamp].csv`
- `ensemble_stacking_predictions_[timestamp].csv` (if available)
- `inference_report_[timestamp].joblib`

### CSV File Structure:
Each CSV file contains:
- `PERSON_ID`: Player identification number
- `SEASON_ID`: '2025-26' (prediction season)
- `INPUT_SEASON_ID`: '2024-25' (season used for prediction)
- All 21 predicted statistics (next_Points, next_FTM, etc.)

### Usage:
These CSV files can be directly loaded into the frontend for visualization and comparison of different model predictions for the 2025-26 NBA season.